In [1]:
# import required libraries

import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from scipy import ndimage
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, Flatten, Dropout, Activation,MaxPooling2D
from keras.models import Sequential
import scipy.io

In [2]:
# import the csv file to see how it looks like
coords_data = pd.read_csv(r"C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\new_annotated_coordinates_all.csv", header = None)

In [3]:
coords_data.head()

,0,1,2,3,4
0,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,42,45,61,40
1,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,40,84,65,41
2,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,34,119,76,36
3,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,71,430,45,50
4,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,59,614,35,46


In [4]:
# size of the dataset
coords_data.shape

(11042, 5)

In [5]:
# add column names
coords_data.columns = ['Image name', 'top_x', 'top_y', 'width', 'height']

In [6]:
coords_data.head()

,Image name,top_x,top_y,width,height
0,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,42,45,61,40
1,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,40,84,65,41
2,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,34,119,76,36
3,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,71,430,45,50
4,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,59,614,35,46


In [7]:
coords_data['Image name'].unique()

array(['190814_djifc6310s_Aug14_SW_row_11_range_2_rep_4_isref_1.png',
       '190814_djifc6310s_Aug14_SW_row_22_range_5_rep_5_isref_1.png',
       '190816_djifc6310s_Aug16_NE_row_16_range_4_rep_7.png',
       '190816_djifc6310s_Aug16_NE_row_41_range_6_rep_7.png',
       '190816_djifc6310s_Aug16_SW_row_11_range_2_rep_3_isref_1.png',
       '190816_djifc6310s_Aug16_SW_row_22_range_5_rep_4_isref_1.png',
       '190820_djifc6310s_Aug20_NE_row_16_range_4_rep_5.png',
       '190820_djifc6310s_Aug20_NE_row_41_range_6_rep_7.png',
       '190820_djifc6310s_Aug20_SW_row_11_range_2_rep_7_isref_1.png',
       '190820_djifc6310s_Aug20_SW_row_22_range_5_rep_4_isref_1.png',
       '190822_djifc6310s_Aug22_NE_row_16_range_4_rep_8.png',
       '190822_djifc6310s_Aug22_NE_row_41_range_6_rep_7.png',
       '190822_djifc6310s_Aug22_SW_row_11_range_2_rep_7_isref_1.png',
       '190822_djifc6310s_Aug22_SW_row_22_range_5_rep_3_isref_1.png',
       '190823_djifc6310_Aug23_NE_row_16_range_4_rep_5.png',
       

In [8]:
len(coords_data['Image name'].unique())

323

In [1]:
# test image path

image_path = r"C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\unfiltered_test"

In [10]:
# print all the image names
image_names = os.listdir(image_path)

In [11]:
image_names

['190812_djifc6310s_Aug12_NE_row_16_range_4_rep_10.png',
 '190812_djifc6310s_Aug12_NW_row_17_range_4_rep_2_isref_1.png',
 '190812_djifc6310s_Aug12_NW_row_32_range_6_rep_6_isref_1.png',
 '190812_djifc6310s_Aug12_SE_row_21_range_3_rep_6.png',
 '190812_djifc6310s_Aug12_SE_row_32_range_4_rep_3.png',
 '190812_djifc6310s_Aug12_SW_row_11_range_2_rep_7_isref_1.png',
 '190812_djifc6310s_Aug12_SW_row_22_range_5_rep_3_isref_1.png',
 '190812_djifc6310s_Aug12_SW_row_31_range_2_rep_6_isref_0.png',
 '190812_djifc6310s_Aug12_SW_row_32_range_5_rep_2_isref_0.png',
 '190812_djifc6310s_Aug12_SW_row_7_range_6_rep_9_isref_0.png',
 '190814_djifc6310s_Aug14_NE_row_16_range_4_rep_8.png',
 '190814_djifc6310s_Aug14_NE_row_18_range_4_rep_8.png',
 '190814_djifc6310s_Aug14_NE_row_1_range_1_rep_2.png',
 '190814_djifc6310s_Aug14_NE_row_21_range_2_rep_4.png',
 '190814_djifc6310s_Aug14_NE_row_33_range_2_rep_5.png',
 '190814_djifc6310s_Aug14_NE_row_39_range_1_rep_3.png',
 '190814_djifc6310s_Aug14_NE_row_41_range_6_rep_2

In [12]:
# image_names = image_names[:-3]

In [13]:
# how many images in test set
len(image_names)

160

#### Step - 1: Extract the coordinates for each image and count how many tassels are there in each image

In [14]:
# Look at the csv file shared
coords_data.head()

,Image name,top_x,top_y,width,height
0,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,42,45,61,40
1,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,40,84,65,41
2,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,34,119,76,36
3,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,71,430,45,50
4,190814_djifc6310s_Aug14_SW_row_11_range_2_rep_...,59,614,35,46


In [15]:
# how many coords in each image? 
# These are the true counts
true_counts_df = pd.DataFrame(coords_data.groupby('Image name').size()).reset_index()

In [16]:
# see the dataset
true_counts_df.head()

,Image name,0
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_...,32
1,190812_djifc6310s_Aug12_NE_row_18_range_4_rep_...,25
2,190812_djifc6310s_Aug12_NE_row_33_range_2_rep_...,51
3,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,36
4,190812_djifc6310s_Aug12_NW_row_22_range_2_rep_...,38


In [17]:
# shape of the dataframe
true_counts_df.shape

(323, 2)

In [18]:
# select the images in the test folder only
test_true_counts = true_counts_df.loc[true_counts_df['Image name'].isin(image_names)].reset_index(drop = True)

In [19]:
# take a look at the head
test_true_counts.head()

,Image name,0
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_...,32
1,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,36
2,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
3,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
4,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19


In [20]:
# how many images here?
test_true_counts.shape


(160, 2)

In [21]:
# rename the column 0 as true counts
test_true_counts.columns = ['Image name', "True count"]

In [22]:
# view the first few observations
test_true_counts

,Image name,True count
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_...,32
1,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,36
2,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53
3,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21
4,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19
...,...,...
155,190905_djifc6310s_Sept5_SE_row_8_range_2_rep_3...,28
156,190905_djifc6310s_Sept5_SW_row_10_range_4_rep_...,41
157,190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...,24
158,190905_djifc6310s_Sept5_SW_row_22_range_6_rep_...,23


In [23]:
image_names

['190812_djifc6310s_Aug12_NE_row_16_range_4_rep_10.png',
 '190812_djifc6310s_Aug12_NW_row_17_range_4_rep_2_isref_1.png',
 '190812_djifc6310s_Aug12_NW_row_32_range_6_rep_6_isref_1.png',
 '190812_djifc6310s_Aug12_SE_row_21_range_3_rep_6.png',
 '190812_djifc6310s_Aug12_SE_row_32_range_4_rep_3.png',
 '190812_djifc6310s_Aug12_SW_row_11_range_2_rep_7_isref_1.png',
 '190812_djifc6310s_Aug12_SW_row_22_range_5_rep_3_isref_1.png',
 '190812_djifc6310s_Aug12_SW_row_31_range_2_rep_6_isref_0.png',
 '190812_djifc6310s_Aug12_SW_row_32_range_5_rep_2_isref_0.png',
 '190812_djifc6310s_Aug12_SW_row_7_range_6_rep_9_isref_0.png',
 '190814_djifc6310s_Aug14_NE_row_16_range_4_rep_8.png',
 '190814_djifc6310s_Aug14_NE_row_18_range_4_rep_8.png',
 '190814_djifc6310s_Aug14_NE_row_1_range_1_rep_2.png',
 '190814_djifc6310s_Aug14_NE_row_21_range_2_rep_4.png',
 '190814_djifc6310s_Aug14_NE_row_33_range_2_rep_5.png',
 '190814_djifc6310s_Aug14_NE_row_39_range_1_rep_3.png',
 '190814_djifc6310s_Aug14_NE_row_41_range_6_rep_2

In [24]:
# save this dataframe for future use
test_true_counts.to_csv('true_counts_test_only.csv', index = False)

#### Step - 2: Preprocessing for test set

In [25]:
# get the trained model
model = keras.models.load_model('newdata_finetunned_s5.h5')


# summary of the model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0   

In [26]:
# Store the images as downsized numpy arrays so that they can be used later for predictions

In [27]:
# get the image path of each image

# path to image directory
image_path = r"C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\unfiltered_gray_test"

# empty list to capture paths
all_new_image_paths = []
for image in image_names:
    # join these two paths
    joined_path = os.path.join(image_path, image)
    
#     append these to the empty list
    all_new_image_paths.append(joined_path)

In [28]:
# length of the paths
len(all_new_image_paths)

160

In [29]:
# store these as numpy arrays after resizing as original images

# specify a path to store these
store_test_np_files = r"C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\numpy test gray images"

# store the size of each image
img_size = []
all_img_names = []
for img_pth in all_new_image_paths:
#     read the image
    read_img = plt.imread(img_pth)
#     resize the image
#     resized_img = resize(read_img, (read_img.shape[0], read_img.shape[1]), 
#                        anti_aliasing = True)
#     append this shape in the empty list
    img_size.append(read_img.shape)
#     give this new image a name
    img_name = (img_pth.split("\\")[-1]).split(".")[0]
#     save the file as np
    np.save(store_test_np_files + "\\" + img_name + "_resized.npy", read_img)
#     save all image names
    all_img_names.append(img_name)
    

In [30]:
# load the np files
np_path = store_test_np_files

# list of names in this
all_np_files = os.listdir(np_path)

# join all the paths
all_joined_paths = []
for np_file in all_np_files:
#     join the path
    joined_path = os.path.join(np_path, np_file)
#     append the path
    all_joined_paths.append(joined_path)

In [31]:
# define a function to get the predictions for each sub image in an image file

def prediction_on_test_data(selected_file, stride = 8, kernel_size = 32):
# load the image data file
    load_image = np.load(store_test_np_files + "\\"+ selected_file + "_resized.npy")
    
    # store the new images
    store_new_path = r"C:\Users\15739\Desktop\UNL - all\Summer 2022\Manuscript\Image_set_1\test set down_sized"
    
    # append the subwindows together

    # get the subwindows
    sub_windows = []

    # get the image path
    all_img_paths = []

    # get the image height
    img_height = load_image.shape[0]
    # get the image weight
    img_width = load_image.shape[1]
    
    # start counter to keep moving
    counter = 0

    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = load_image[i: i + kernel_size, j : j + kernel_size,:]
    #     append the sub window images
            sub_windows.append(sub_window)
    #     increase the counter
            counter = counter + 1
    #     store the images
            each_path = (store_new_path + "\\" + selected_file + "_" + str(counter) + ".jpg")
            plt.imsave(each_path, sub_window, cmap = "gray")
    #         append each path
            all_img_paths.append(each_path)

    # put the image paths in a dataframe
    test_df_img_0 = pd.DataFrame(all_img_paths, columns = ["Image_path"])

    # now, to do the prediction, define then test data generator
    test_gen = ImageDataGenerator(rescale = 1./255.)

    # use flow from data frame to define the test datagenerator
    Test_generator = test_gen.flow_from_dataframe(dataframe = test_df_img_0, directory = None, x_col = "Image_path", 
                                                y_col = None, target_size = (32,32), class_mode = None)

    # now, to get the predictions, can use this as it is
    test_image0_prediction = model.predict(Test_generator)
    
    return(test_image0_prediction, selected_file, (img_height,img_width))





In [32]:
# catch the predicted values for all subimages per each image
predicted_values_all = []

# get the corresponding image name
Image_name = []

# get the image shapes
Image_shapes = []

# do this for all the files we have (these are stored in unique_file_names)
for file in all_img_names:
    return_everything = prediction_on_test_data(file)
#     get the predictions appended 
    predicted_counts = return_everything[0]
    predicted_values_all.append(predicted_counts)
    
#     append the image names
    image_name = return_everything[1]
    Image_name.append(image_name)
    
#     Append the image sizes
    image_shape = return_everything[2]
    Image_shapes.append(image_shape)

Found 4488 validated image filenames.
Found 3540 validated image filenames.
Found 3540 validated image filenames.
Found 4356 validated image filenames.
Found 4356 validated image filenames.
Found 3540 validated image filenames.
Found 3540 validated image filenames.
Found 3540 validated image filenames.
Found 3540 validated image filenames.
Found 3540 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3906 validated image filenames.
Found 3052 validated image filenames.
Found 3052 validated image filenames.
Found 3052 validated image filenames.
Found 3813 validated image filenames.
Found 3813 validated image filenames.
Found 3813 validated image filenames.
Found 3813 validated image filenames.
Found 3052 validated image filenames.
Found 3052 validated image filenames.
Found 3052 v

In [33]:
# define a function for getting the final predictions
def get_the_final_preds(pred_vals, im_shape, stride_size = 8, kernel = 32):
    # get the corresponding shape of the image

    # height
    image_height = im_shape[0]
    
    #width
    image_width = im_shape[1]
    
    #create the density and the count empty maps
    
    # density map
    Density_map = np.zeros((image_height, image_width))

    # counts map
    counts_map = np.zeros((image_height, image_width))
    
    # now, for every window, we will keep adding the values together and also add the counts
    counter = 0
#     need a counter to move into each predicted value in the pred values list
    for ii in range(0, image_height, stride_size):
        for jj in range(0, image_width, stride_size):
#         operations for density map
#             get the window of interest
            new_window = Density_map[ii:ii + kernel,jj:jj+kernel]
#     fill each with the value c_k
            counts_window = np.full((new_window.shape[0], new_window.shape[1]), pred_vals[counter])
#     get the shapes of this new window
            cw_height = counts_window.shape[0]
            cw_width = counts_window.shape[1]
#         Do c_k/r_2
            counts_window_new = counts_window/(cw_height*cw_width)
#     This is the value in the window now
            value_window = counts_window_new
#     place the values in the corrsponding area of the density map
            Density_map[ii:ii + kernel,jj:jj+kernel] = new_window + value_window

#         Let's now focus on capturing the counts of the windows
            new_window_c = counts_map[ii:ii + kernel,jj:jj+kernel]
#     get the counts area
            count = np.ones((new_window_c.shape[0], new_window_c.shape[1]))
#     keep adding the counts to reflect the addition of densities
            counts_map[ii:ii + kernel,jj:jj+kernel] = new_window_c + count
#     increase the counter
            counter = counter + 1
            
#         get the normalized count
    normalized_counts = np.divide(Density_map, counts_map)
    
#     entire count on the test set
    pred_on_test = np.sum(normalized_counts)
    
#     return the predicted value
    return(pred_on_test, normalized_counts)

In [34]:
# run it for all the predictions obtained
# store all the predictions
norm_counts = []
predicted_values = []
for i in range(len(Image_shapes)):
    pred_vals = predicted_values_all[i]
    im_shape = Image_shapes[i]
#     run the defined function
    predictions_computation = get_the_final_preds(pred_vals, im_shape)
#     append the results
    predicted_values.append(predictions_computation[0])
    norm_counts.append(predictions_computation[1])

In [35]:
predicted_values

[23.4625577850005,
 14.314059924659666,
 38.94968484129391,
 25.63974844912999,
 24.43258284923741,
 30.808605637690707,
 22.799963866627934,
 33.35086035903546,
 19.567050452862258,
 40.158262850911434,
 28.521920232248842,
 17.49183477813535,
 34.87272106325558,
 47.97921160377523,
 48.24512797697772,
 1.899352776334159,
 15.946931432130885,
 19.788660478035656,
 37.696866213136616,
 19.472240111828214,
 30.803955074783076,
 24.03455428274638,
 21.773083796460295,
 19.919158802982828,
 11.789047382228796,
 27.789567566476357,
 42.955129057251284,
 18.403179653794062,
 35.81019640769837,
 40.03124948709967,
 36.957375141834035,
 28.38426007720942,
 14.42509512303108,
 68.3894442653538,
 23.378718518063387,
 44.34753887784834,
 20.5369434766934,
 18.541123135700126,
 8.29736953171488,
 28.447858649570453,
 37.15059214519247,
 33.97259431913562,
 36.94909987777095,
 41.602259246031686,
 36.588757409236166,
 39.27259075764015,
 32.48273330905452,
 17.64643757281556,
 37.15873564681076,
 

In [36]:
# d = {'Image name': Image_name, 'Predicted value': pred_data}
pred_data  = list(zip(Image_name, predicted_values))

In [37]:
pred_data = pd.DataFrame(pred_data)

In [38]:
pred_data.columns = ['Image name', 'Predicted value']

In [39]:
pred_data

,Image name,Predicted value
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_10,23.462558
1,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,14.314060
2,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,38.949685
3,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_6,25.639748
4,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_3,24.432583
...,...,...
155,190905_djifc6310s_Sept5_SE_row_8_range_2_rep_3,17.110603
156,190905_djifc6310s_Sept5_SW_row_10_range_4_rep_...,31.306448
157,190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...,27.938836
158,190905_djifc6310s_Sept5_SW_row_22_range_6_rep_...,26.795729


In [40]:
# merge the true and predicted counts
final_data = pd.concat((test_true_counts, pred_data.iloc[:,1]), axis = 1)

In [41]:
final_data

,Image name,True count,Predicted value
0,190812_djifc6310s_Aug12_NE_row_16_range_4_rep_...,32,23.462558
1,190812_djifc6310s_Aug12_NW_row_17_range_4_rep_...,36,14.314060
2,190812_djifc6310s_Aug12_NW_row_32_range_6_rep_...,53,38.949685
3,190812_djifc6310s_Aug12_SE_row_21_range_3_rep_...,21,25.639748
4,190812_djifc6310s_Aug12_SE_row_32_range_4_rep_...,19,24.432583
...,...,...,...
155,190905_djifc6310s_Sept5_SE_row_8_range_2_rep_3...,28,17.110603
156,190905_djifc6310s_Sept5_SW_row_10_range_4_rep_...,41,31.306448
157,190905_djifc6310s_Sept5_SW_row_11_range_2_rep_...,24,27.938836
158,190905_djifc6310s_Sept5_SW_row_22_range_6_rep_...,23,26.795729


In [42]:
# MAE for the new data
MAE = np.mean(abs(final_data['True count'] - final_data['Predicted value']))
MAE

6.790284303179902

In [43]:
# save the pred and true counts
final_data.to_csv('predicted_counts_finetuned_s5.csv', index = False)